In [3]:
import torch
import numpy as np
from torch import optim
from scipy.sparse import csr_matrix
from torch.utils.data import Dataset, DataLoader
import scipy.sparse as sp
from time import time
from torch import nn
import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [4]:
import shutil
shutil.copy('../input/lightgcn-test/LightGCN-PyTorch-master/data/gowalla/train.txt', './')
shutil.copy('../input/lightgcn-test/LightGCN-PyTorch-master/data/gowalla/test.txt', './')

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
'''
def set_seed(seed):
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.manual_seed(seed)
set_seed(2020)
'''

In [6]:
class Dataloader(Dataset):
    def __init__(self, path='./'):
        self.path = './'
        trainUniqueUsers, trainItem, trainUser = [], [], []
        testUniqueUsers, testItem, testUser = [], [], []
        n_user, m_item = 0, 0
        train_file = path + 'train.txt'
        test_file = path + 'test.txt'
        trainDataSize, testDataSize = 0, 0
        with open(train_file) as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    trainUniqueUsers.append([uid])
                    trainUser.extend([uid] * len(items))
                    trainItem.extend(items)
                    m_item = max(m_item, max(items))
                    n_user = max(n_user, uid)
                    trainDataSize += len(items)

        self.trainUniqueUsers = np.array(trainUniqueUsers)
        self.trainUser = np.array(trainUser)
        self.trainItem = np.array(trainItem)
        self.trainDataSize = trainDataSize
        
        with open(test_file) as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    testUser.extend([uid] * len(items))
                    testItem.extend(items)
                    m_item = max(m_item, max(items))
                    n_user = max(n_user, uid)
                    testDataSize += len(items)

        self.testUniqueUsers = np.array(testUniqueUsers)
        self.testUser = np.array(testUser)
        self.testItem = np.array(testItem)
        self.testDataSize = testDataSize
        
        self.n_user = n_user + 1
        self.m_item = m_item + 1

        print(f"{self.trainDataSize} interactions for training")
        print(f"{self.testDataSize} interactions for testing")
        print(f"{self.n_user} users, {self.m_item} items")
        print(f"Gowalla Sparsity : {(self.trainDataSize + self.testDataSize) / self.n_user / self.m_item}")
        self.UserItemNet = csr_matrix((np.ones(len(self.trainUser)), (self.trainUser, self.trainItem)),
                                          shape=(self.n_user, self.m_item))
        self.allPos = self.getAllPos(list(range(self.n_user)))
        self.users_D = np.array(self.UserItemNet.sum(axis=1)).squeeze()
        self.users_D[self.users_D == 0.] = 1.
        self.items_D = np.array(self.UserItemNet.sum(axis=0)).squeeze()
        self.items_D[self.items_D == 0.] = 1.
        self.Graph = None
        self.testDict = self.getTestDict()
    

    def getAllPos(self, users):
        posItems = []
        for user in users:
            posItems.append(self.UserItemNet[user].nonzero()[1])
        return posItems
    
    #采样
    def uniformSample(self):
        S = []
        users = np.random.randint(0, self.n_user, self.trainDataSize)
        for user in users:
            posItemList = self.allPos[user]
            if len(posItemList) == 0:
                continue
            posIndex = np.random.randint(0, len(posItemList))
            posItem = posItemList[posIndex]
            while True:
                negItem = np.random.randint(0, self.m_item)
                if negItem in posItemList:
                    continue
                else:
                    break
            S.append([user, posItem, negItem])
        return np.array(S)
    
    
    def _convert_sp_mat_to_sp_tensor(self, X):
        coo = X.tocoo().astype(np.float32)
        row = torch.Tensor(coo.row).long()
        col = torch.Tensor(coo.col).long()
        index = torch.stack([row, col])
        data = torch.FloatTensor(coo.data)
        return torch.sparse.FloatTensor(index, data, torch.Size(coo.shape))
    
    def getSparseGraph(self):
        if self.Graph is None:
            try:
                pre_adj_mat = sp.load_npz(self.path + '/s_pre_adj_mat.npz')
                print("successfully loaded adjacency matrix")
                norm_adj = pre_adj_mat
            except :
                print("generating adjacency matrix")
                s = time()
                adj_mat = sp.dok_matrix((self.n_user + self.m_item, self.n_user + self.m_item), dtype=np.float32)
                adj_mat = adj_mat.tolil()
                R = self.UserItemNet.tolil()
                adj_mat[:self.n_user, self.n_user:] = R
                adj_mat[self.n_user:, :self.n_user] = R.T
                adj_mat = adj_mat.todok()
                # adj_mat = adj_mat + sp.eye(adj_mat.shape[0])
                
                rowsum = np.array(adj_mat.sum(axis=1))
                d_inv = np.power(rowsum, -0.5).flatten()
                d_inv[np.isinf(d_inv)] = 0.
                d_mat = sp.diags(d_inv)
                
                norm_adj = d_mat.dot(adj_mat)
                norm_adj = norm_adj.dot(d_mat)
                norm_adj = norm_adj.tocsr()
                end = time()
                print(f"costing {end-s}s, saved norm_mat...")
                sp.save_npz(self.path + '/s_pre_adj_mat.npz', norm_adj)
            
            self.Graph = self._convert_sp_mat_to_sp_tensor(norm_adj)
            self.Graph = self.Graph.coalesce().to(device)
            print("don't split the matrix")
            
        return self.Graph
    
    def getTestDict(self):
        test_data = {}
        for i, item in enumerate(self.testItem):
            user = self.testUser[i]
            if test_data.get(user):
                test_data[user].append(item)
            else:
                test_data[user] = [item]
        return test_data

In [7]:
class BPRLoss:
    def __init__(self, model, lr=1e-3, weight_decay=1e-4):
        self.model = model
        self.weight_decay = weight_decay
        self.lr = lr
        self.opt = optim.Adam(model.parameters(), lr=self.lr)
    
    def stageOne(self,users, pos, neg):
        loss, reg_loss = self.model.bpr_loss(users, pos, neg)
        reg_loss = reg_loss*self.weight_decay
        loss = loss + reg_loss

        self.opt.zero_grad()
        loss.backward()
        self.opt.step()

        return loss.cpu().item()

In [8]:
class LightGCN(nn.Module):
    def __init__(self, dataset, latent_dim=64, n_layers=3):
        super(LightGCN, self).__init__()
        self.dataset = dataset
        
        self.num_users = self.dataset.n_user
        self.num_items = self.dataset.m_item
        self.latent_dim = latent_dim
        self.n_layers = n_layers
        self.embedding_user = torch.nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.latent_dim)
        self.embedding_item = torch.nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.latent_dim)
        nn.init.normal_(self.embedding_user.weight, std=0.1)
        nn.init.normal_(self.embedding_item.weight, std=0.1)
        
        self.f = nn.Sigmoid()
        self.Graph = self.dataset.getSparseGraph()
        print(f"lgn is already to go")
    
    def computer(self):
        users_emb = self.embedding_user.weight
        items_emb = self.embedding_item.weight
        g = self.Graph
        all_emb = torch.cat([users_emb, items_emb])
        embs = [all_emb]
        
        for layer in range(self.n_layers):
            all_emb = torch.sparse.mm(g, all_emb)
            embs.append(all_emb)
            
        embs = torch.stack(embs, dim=1)

        light_out = torch.mean(embs, dim=1)

        users, items = torch.split(light_out, [self.num_users, self.num_items])
        return users, items

    def getEmbedding(self, users, pos_items, neg_items):
        all_users, all_items = self.computer()
        users_emb = all_users[users]
        pos_emb = all_items[pos_items]
        neg_emb = all_items[neg_items]
        users_emb_ego = self.embedding_user(users)
        pos_emb_ego = self.embedding_item(pos_items)
        neg_emb_ego = self.embedding_item(neg_items)
        return users_emb, pos_emb, neg_emb, users_emb_ego, pos_emb_ego, neg_emb_ego
    
    def bpr_loss(self, users, pos, neg):
        (users_emb, pos_emb, neg_emb, 
        userEmb0,  posEmb0, negEmb0) = self.getEmbedding(users.long(), pos.long(), neg.long())
        reg_loss = (1/2)*(userEmb0.norm(2).pow(2) + 
                         posEmb0.norm(2).pow(2)  +
                         negEmb0.norm(2).pow(2))/float(len(users))
        pos_scores = torch.mul(users_emb, pos_emb)
        pos_scores = torch.sum(pos_scores, dim=1)
        neg_scores = torch.mul(users_emb, neg_emb)
        neg_scores = torch.sum(neg_scores, dim=1)
        
        loss = torch.mean(torch.nn.functional.softplus(neg_scores - pos_scores))
        return loss, reg_loss
    
    def forward(self, users, items):
        all_users, all_items = self.computer()
        users_emb = all_users[users]
        items_emb = all_items[items]
        y_hat = torch.mul(users_emb, items_emb)
        y_hat = y_hat.sum(dim=1)
        return y_hat
    
    def getUserRating(self, users):
        all_users, all_items = self.computer()
        users_emb = all_users[users.long()]
        items_emb = all_items
        rating = self.f(torch.matmul(users_emb, items_emb.t()))
        return rating

In [9]:
def get_minibatch(batch_size, *tensors):
    if len(tensors) == 1:
        tensor = tensors[0]
        for i in range(0, len(tensor), batch_size):
            yield tensor[i: i+batch_size]
    else:
        for i in range(0, len(tensors[0]), batch_size):
            yield tuple(x[i: i+batch_size] for x in tensors)
    

In [10]:
def train(dataset, model, loss, epochs, batch_size=64):
    
    all_start = time()
    
    for epoch in range(epochs):
        start = time()
        model.train()
        S = dataset.uniformSample()
        users = torch.tensor(S[:, 0]).long().to(device)
        posItems = torch.tensor(S[:, 1]).long().to(device)
        negItems = torch.tensor(S[:, 2]).long().to(device)
        
        new_idx = np.random.permutation(len(users))
        
        users = users[new_idx]
        posItems = posItems[new_idx]
        negItems = negItems[new_idx]
        
        total_batch = len(users) // batch_size + 1
        avg_loss = 0
        
        for (idx, (user, posItem, negItem)) in enumerate(get_minibatch(batch_size, users, posItems, negItems)):
            avg_loss += loss.stageOne(user, posItem, negItem)
        
        avg_loss /= total_batch
        end = time()
        print(f'Epoch {epoch+1:^5d}/{epochs:^5d}: train loss {avg_loss:.5f}, cost {end-start:.1f}s')
        if (epoch + 1) % 100 == 0:
            torch.save(model.state_dict(), f'lgn-{epoch+1}-{avg_loss:.3f}.pkl')
            K = 20
            test_batch_size = 100
            test(dataset, model, test_batch_size, K)
            
    print(f'All cost {time() - all_start: .1f}s')

In [11]:
dataset = Dataloader()

In [12]:
lgn_model = LightGCN(dataset).to(device)

In [13]:
loss = BPRLoss(lgn_model)

In [14]:
def NDCG(test_data, r, K):
    assert len(test_data) == len(r)
    pred_data = r[:, :K]
    test_matrix = np.zeros((len(pred_data), K))
    for i, items in enumerate(test_data):
        length = K if K <= len(items) else len(items)
        test_matrix[i, :length] = 1
    idcg = np.sum(test_matrix * 1.0 / np.log(np.arange(2, K+2)), axis=1)
    dcg = np.sum(pred_data * 1.0 / np.log(np.arange(2, K+2)), axis=1)
    idcg[idcg == 0.] = 1.
    ndcg = dcg / idcg
    ndcg[np.isnan(ndcg)] = 0.
    return np.sum(ndcg)

In [15]:
def test(dataset, model, batch_size, K):
    testDict = dataset.testDict
    model.eval()
    precision, recall, ndcg = 0.0, 0.0, 0.0
    user_cnt = 0
    with torch.no_grad():
        users = list(testDict.keys())
        for batch_users in get_minibatch(batch_size, users):
            user_cnt += len(batch_users)
            allPos = dataset.getAllPos(batch_users)
            groundTrue = [testDict[u] for u in batch_users]
            
            batch_users_gpu = torch.Tensor(batch_users).long()
            batch_users_gpu = batch_users_gpu.to(device)
            
            rating = model.getUserRating(batch_users_gpu)
            exclude_users = []
            exclude_items = []
            for i, items in enumerate(allPos):
                exclude_users.extend(len(items) * [i])
                exclude_items.extend(items)
            
            rating[exclude_users, exclude_items] = - (1<<10)
            _, rating_K = torch.topk(rating, k=K)
            rating_K = rating_K.cpu()

            r, recall_n  = [], []
            precision_n = K
            for i, y in enumerate(groundTrue):
                y_hat = rating_K[i]
                pred = list(map(lambda x: x in y, y_hat))
                pred = np.array(pred).astype('float')
                r.append(pred)
                recall_n.append(len(y))
            r = np.array(r).astype('float')
            ndcg += NDCG(groundTrue, r, K)
            r = r[:, :K].sum(1)
            recall_n = np.array(recall_n).astype('float')
            precision += np.sum(r) / precision_n
            recall += np.sum(r / recall_n)
            
            
        precision /= user_cnt
        recall /= user_cnt
        ndcg /= user_cnt
    print(f'Test precision: {precision:.4f}, recall: {recall:.4f}, ndcg: {ndcg:.4f}')
            

In [16]:
#batch_size = 2048
batch_size = 2048
epochs = 1000
train(dataset, lgn_model, loss, epochs, batch_size)